In [ ]:
import pandas as pd
import numpy as np

def evaluate_provider_combinations(df):

    # Ensure columns exist
    required_cols = [
        "providertaxid", "providernpi", 
        "sel_category", "paid_amount_bucket",
        "audits", "findings", "nofindings",
        "op_amount", "dispute_count", "overturn_count"
    ]

    for col in required_cols:
        if col not in df.columns:
            df[col] = 0   # missing columns default to 0 so algorithm won't break

    # -------------------------------------------------------
    # Step 1: Derived metrics (dynamic)
    # -------------------------------------------------------
    df["hitrate"] = df["findings"] / df["audits"].replace(0, np.nan)
    df["dispute_rate"] = df["dispute_count"] / df["audits"].replace(0, np.nan)
    df["overturn_rate"] = df["overturn_count"] / df["findings"].replace(0, np.nan)

    # -------------------------------------------------------
    # Step 2: Dynamic scoring within each provider
    # -------------------------------------------------------
    def score_per_provider(group):

        metric_cols = ["hitrate", "op_amount", "audits"]

        # Compute z-scores only if variation exists
        for col in metric_cols:
            if group[col].nunique() > 1:
                group[col + "_z"] = (group[col] - group[col].mean()) / group[col].std()
            else:
                group[col + "_z"] = 0

        # Weighted final score (can adjust weights anytime)
        group["score"] = (
            0.5 * group["hitrate_z"] +
            0.3 * group["op_amount_z"] +
            0.2 * group["audits_z"]
        )

        return group

    df = df.groupby(["providertaxid", "providernpi"], group_keys=False).apply(score_per_provider)

    # -------------------------------------------------------
    # Step 3: Classification into buckets
    # -------------------------------------------------------
    def classify(row):
        if row["audits"] < 15:
            return "Insufficient Data – low audit sample"
        if row["score"] > 0.7:
            return "High Performing"
        if 0.4 <= row["score"] <= 0.7:
            return "Moderate Performing"
        if row["score"] < 0.4:
            return "Low Performing"
    
    df["performance_label"] = df.apply(classify, axis=1)

    # -------------------------------------------------------
    # Step 4: Insight Generation
    # -------------------------------------------------------
    def create_insight(row):

        if row["performance_label"].startswith("Insufficient"):
            return (
                f"Only {row['audits']} audits — not enough data to judge performance "
                f"in {row['sel_category']} – {row['paid_amount_bucket']}."
            )

        if row["performance_label"] == "High Performing":
            return (
                f"Strong performance in {row['sel_category']} – {row['paid_amount_bucket']}: "
                f"Hitrate {row['hitrate']:.2%}, OP ${row['op_amount']:.2f}. "
                f"Among the best categories for this provider."
            )

        if row["performance_label"] == "Moderate Performing":
            return (
                f"Average performance in {row['sel_category']} – {row['paid_amount_bucket']}. "
                f"Hitrate and OP are close to provider's overall average."
            )

        if row["performance_label"] == "Low Performing":
            return (
                f"Poor performance in {row['sel_category']} – {row['paid_amount_bucket']}: "
                f"Hitrate {row['hitrate']:.2%}, OP ${row['op_amount']:.2f}. "
                f"Much lower than other categories for this provider. "
                f"Recommended for deprioritization/exclusion."
            )

    df["insight"] = df.apply(create_insight, axis=1)

    return df


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

def evaluate_provider_performance(df):
    # Compute basic metrics
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['overpayment'] / df['audits']
    df['dispute_rate'] = df['disputes'] / df['audits']
    df['overturn_rate'] = df['overturns'] / df['audits']

    # Remove combinations with <15 audits ONLY FROM EVALUATION
    df['eligible'] = df['audits'] >= 15

    # Standardize metrics provider-wise (TIN + NPI level)
    def provider_zscores(x):
        metrics = ['hitrate','op_per_audit','overturn_rate']
        for m in metrics:
            x[f'{m}_z'] = zscore(x[m], ddof=1) if x[m].nunique() > 1 else 0
        return x

    df = df.groupby(['providertaxid','providernpi']).apply(provider_zscores)

    # Overall performance score (higher = better)
    df['performance_score'] = (
        df['hitrate_z'] * 0.5 +
        df['op_per_audit_z'] * 0.3 +
        df['overturn_rate_z'] * 0.2
    )

    # Insights (dynamic – provider-wise quartiles)
    def assign_insight(x):
        if len(x[x['eligible']]) == 0:
            x['insight'] = "Not enough audit volume to evaluate"
            return x

        scores = x.loc[x['eligible'], 'performance_score']
        q1, q3 = scores.quantile([0.25, 0.75])

        def label(row):
            if not row['eligible']:
                return "Low audits — excluded from evaluation"

            if row['performance_score'] >= q3:
                return ("Strong performer — This selection category & paid bucket "
                        "consistently shows high hitrate, OP and overturn success "
                        "compared to other combinations for this provider")

            elif row['performance_score'] <= q1:
                return ("Weak performer — This combination underperforms in hitrate, OP "
                        "and overturn rate compared to other categories for the same provider")

            else:
                return ("Moderate performer — Performance is neither high nor low but "
                        "should be monitored relative to other categories")

        x['insight'] = x.apply(label, axis=1)
        return x

    df = df.groupby(['providertaxid','providernpi']).apply(assign_insight)

    return df


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# -------------------------------------------------------------
# 1. MAIN ALGORITHM — AUTO DETECTS WEAK / STRONG COMBINATIONS
# -------------------------------------------------------------
def evaluate_provider_performance(df):

    # ---- Basic computed metrics ----
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['overpayment'] / df['audits']
    df['dispute_rate'] = df['disputes'] / df['audits']
    df['overturn_rate'] = df['overturns'] / df['audits']

    # ---- Exclude low audit volume (<15) from scoring ----
    df['eligible'] = df['audits'] >= 15

    # ---- Compute z-scores within each provider ----
    def provider_zscores(x):
        metrics = ['hitrate', 'op_per_audit', 'overturn_rate']
        for m in metrics:
            if x[m].nunique() > 1:
                x[f'{m}_z'] = zscore(x[m], ddof=1)
            else:
                x[f'{m}_z'] = 0
        return x

    df = df.groupby(['providertaxid', 'providernpi']).apply(provider_zscores)

    # ---- Weighted dynamic performance score ----
    df['performance_score'] = (
        df['hitrate_z'] * 0.5 +
        df['op_per_audit_z'] * 0.3 +
        df['overturn_rate_z'] * 0.2
    )

    # -------------------------------------------------------------
    # 2. Assign insight category based on provider-level distribution
    # -------------------------------------------------------------
    def assign_insight(x):
        # If provider has zero eligible rows
        if len(x[x['eligible']]) == 0:
            x['insight'] = "Not enough audit volume to evaluate"
            return x

        eligible_scores = x.loc[x['eligible'], 'performance_score']
        q1, q3 = eligible_scores.quantile([0.25, 0.75])

        def label(row):
            if not row['eligible']:
                return "Low audits (<15) — excluded from evaluation"

            if row['performance_score'] >= q3:
                return (
                    "Strong performer — This category shows superior hitrate, "
                    "higher overpayment recovery per audit, and better overturn success "
                    "than other categories for this provider."
                )

            elif row['performance_score'] <= q1:
                return (
                    "Weak performer — Underperforms in hitrate, overpayment recovery, "
                    "and overturn rate compared to other categories of the same provider."
                )

            else:
                return (
                    "Moderate performer — Average performance; neither strong nor weak. "
                    "Monitor for future trend."
                )

        x['insight'] = x.apply(label, axis=1)
        return x

    df = df.groupby(['providertaxid','providernpi']).apply(assign_insight)
    return df


# -------------------------------------------------------------
# 3. REPORT GENERATOR — MANAGER FRIENDLY SUMMARY
# -------------------------------------------------------------
def generate_provider_report(df):

    report_dict = {}
    grouped = df.groupby(['providertaxid', 'providernpi'])

    for (tin, npi), data in grouped:

        # Only eligible categories (audits >=15)
        eligible = data[data['eligible']]

        if eligible.empty:
            report_dict[(tin, npi)] = {
                "top_performers": [],
                "weak_performers": [],
                "summary": f"TIN {tin}, NPI {npi} cannot be evaluated (all categories have <15 audits)."
            }
            continue

        top = eligible[eligible['insight'].str.contains("Strong performer")]
        weak = eligible[eligible['insight'].str.contains("Weak performer")]

        # Build readable summary
        summary_lines = [
            f"Performance Summary for Provider (TIN: {tin}, NPI: {npi})",
            "------------------------------------------------------------"
        ]

        if not top.empty:
            summary_lines.append(
                f"• {len(top)} strong performing categories — Higher hitrate, OP recovery "
                "and overturn pattern compared to provider's other categories."
            )
        else:
            summary_lines.append("• No strong performing categories detected.")

        if not weak.empty:
            summary_lines.append(
                f"• {len(weak)} weak performing categories — These categories perform "
                "significantly below the provider's other categories."
            )
        else:
            summary_lines.append("• No underperforming categories detected.")

        summary_lines.append(f"• Total categories evaluated: {len(eligible)}")
        summary_lines.append("")

        # Store report for this TIN+NPI
        report_dict[(tin, npi)] = {
            "top_performers": top[['sel_category','paid_amount_bucket','audits','hitrate','performance_score','insight']],
            "weak_performers": weak[['sel_category','paid_amount_bucket','audits','hitrate','performance_score','insight']],
            "summary": "\n".join(summary_lines)
        }

    return report_dict


# -------------------------------------------------------------
# 4. RUN THE FULL PIPELINE
# -------------------------------------------------------------
# df = your_dataframe  # <--- Replace with your real data

# evaluated_df = evaluate_provider_performance(df)
# provider_reports = generate_provider_report(evaluated_df)

# NOW YOU CAN ACCESS:
# provider_reports[(TIN, NPI)]["summary"]
# provider_reports[(TIN, NPI)]["top_performers"]
# provider_reports[(TIN, NPI)]["weak_performers"]


In [ ]:
Performance Score =
  0.40 × hitrate
+ 0.35 × op_per_audit
− 0.15 × dispute_on_find
− 0.10 × overturn_on_find


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# ----------------------------------------------------
# STEP 1: METRIC CREATION
# ----------------------------------------------------
def build_metrics(df):

    df = df.copy()

    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['op_amount'] / df['audits']
    df['dispute_on_find'] = df['disputes'] / df['findings']
    df['overturn_on_find'] = df['overturns'] / df['findings']

    return df


# ----------------------------------------------------
# STEP 2: STANDARDIZE WITHIN PEER GROUP
# Peer = same model_band + sel_category
# ----------------------------------------------------
def standardize_peers(df):

    metrics = [
        'hitrate',
        'op_per_audit',
        'dispute_on_find',
        'overturn_on_find'
    ]

    def zscore_group(x):
        for m in metrics:
            if x[m].nunique() > 1:
                x[f'{m}_z'] = zscore(x[m], nan_policy='omit')
            else:
                x[f'{m}_z'] = 0
        return x

    return (
        df.groupby(['model_band', 'sel_category'])
          .apply(zscore_group)
          .reset_index(drop=True)
    )


# ----------------------------------------------------
# STEP 3: COMPOSITE PERFORMANCE SCORE (BUSINESS ALIGNED)
# ----------------------------------------------------
def compute_performance_score(df):

    df['performance_score'] = (
        df['hitrate_z'] * 0.40 +
        df['op_per_audit_z'] * 0.35 -
        df['dispute_on_find_z'] * 0.15 -
        df['overturn_on_find_z'] * 0.10
    )

    return df


# ----------------------------------------------------
# STEP 4: DECISION LOGIC
# ----------------------------------------------------
def assign_decision(df):

    df['decision'] = 'MONITOR'
    df['insight'] = ''

    def classify(group):

        eligible = group[group['audits'] >= 15]

        if eligible.empty:
            group['decision'] = 'NOT EVALUATED'
            group['insight'] = (
                'Audit volume too low across this model band and '
                'selection category to assess performance.'
            )
            return group

        low_cut = eligible['performance_score'].quantile(0.25)
        high_cut = eligible['performance_score'].quantile(0.75)

        for idx, row in group.iterrows():

            if row['audits'] < 15:
                group.loc[idx, ['decision','insight']] = [
                    'NOT EVALUATED',
                    'Insufficient audits to reliably evaluate performance.'
                ]

            elif row['performance_score'] <= low_cut:
                group.loc[idx, ['decision','insight']] = [
                    'EXCLUDE',
                    'Low hitrate and weak value recovery combined with higher '
                    'dispute or overturn behavior relative to peers in the same '
                    'model band and selection category.'
                ]

            elif row['performance_score'] >= high_cut:
                group.loc[idx, ['decision','insight']] = [
                    'KEEP',
                    'Strong audit effectiveness with higher hitrate and overpayment '
                    'recovery, and lower dispute/overturn rates compared to peers.'
                ]

            else:
                group.loc[idx, ['decision','insight']] = [
                    'MONITOR',
                    'Performance is in line with peer combinations; continue monitoring.'
                ]

        return group

    return (
        df.groupby(['model_band', 'sel_category'])
          .apply(classify)
          .reset_index(drop=True)
    )


# ----------------------------------------------------
# STEP 5: FULL PIPELINE
# ----------------------------------------------------
def evaluate_tin_npi_combinations(df):

    df = build_metrics(df)
    df = standardize_peers(df)
    df = compute_performance_score(df)
    df = assign_decision(df)

    return df


# ----------------------------------------------------
# RUN
# ----------------------------------------------------
# df = your_dataframe
# result = evaluate_tin_npi_combinations(df)

# FINAL OUTPUTS:
# result[result['decision'] == 'EXCLUDE']
# result[result['decision'] == 'KEEP']


In [ ]:
import pandas as pd

# ----------------------------------------------------
# STEP 1: METRICS
# ----------------------------------------------------
def create_metrics(df):

    df = df.copy()
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['op_amount'] / df['audits']

    return df


# ----------------------------------------------------
# STEP 2: DYNAMIC LOW THRESHOLDS
# ----------------------------------------------------
def compute_thresholds(df):

    def tag(group):

        hr_cut = group['hitrate'].quantile(0.30)
        op_cut = group['op_per_audit'].quantile(0.30)

        group['low_hitrate'] = group['hitrate'] < hr_cut
        group['low_op'] = group['op_per_audit'] < op_cut

        return group

    return (
        df.groupby(['model_band', 'sel_category'])
          .apply(tag)
          .reset_index(drop=True)
    )


# ----------------------------------------------------
# STEP 3: FINAL DECISION LOGIC
# ----------------------------------------------------
def assign_decision(df):

    df['decision'] = 'MONITOR'
    df['insight'] = ''

    for idx, row in df.iterrows():

        if row['audits'] < 15:
            df.loc[idx, ['decision','insight']] = [
                'NOT EVALUATED',
                'Audit volume below 15; performance not statistically reliable.'
            ]

        elif row['low_hitrate'] and row['low_op']:
            df.loc[idx, ['decision','insight']] = [
                'EXCLUDE',
                'Both hitrate and overpayment recovery are low compared to peer '
                'combinations within the same model band and selection category.'
            ]

        elif not row['low_hitrate'] and not row['low_op']:
            df.loc[idx, ['decision','insight']] = [
                'KEEP',
                'Strong performance with acceptable hitrate and value recovery '
                'relative to peers.'
            ]

        else:
            df.loc[idx, ['decision','insight']] = [
                'MONITOR',
                'Mixed performance — either hitrate or value recovery is acceptable; '
                'continue monitoring.'
            ]

    return df


# ----------------------------------------------------
# STEP 4: FULL PIPELINE
# ----------------------------------------------------
def evaluate_combinations(df):

    df = create_metrics(df)
    df = compute_thresholds(df)
    df = assign_decision(df)

    return df


# ----------------------------------------------------
# RUN
# ----------------------------------------------------
# df = your_dataframe
# result = evaluate_combinations(df)

# EXCLUSION LIST:
# result[result['decision'] == 'EXCLUDE']


In [ ]:
## Fixed leakage

import pandas as pd

# ----------------------------------------------------
# STEP 1: METRIC CREATION
# ----------------------------------------------------
def create_metrics(df):

    df = df.copy()
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['op_amount'] / df['audits']

    return df


# ----------------------------------------------------
# STEP 2: COMPUTE DYNAMIC LOW THRESHOLDS
# ----------------------------------------------------
def compute_thresholds(df):

    def tag(group):

        # Exclude zero rows when computing percentiles
        hr_nonzero = group.loc[group['hitrate'] > 0, 'hitrate']
        op_nonzero = group.loc[group['op_per_audit'] > 0, 'op_per_audit']

        hr_cut = hr_nonzero.quantile(0.30) if not hr_nonzero.empty else 0
        op_cut = op_nonzero.quantile(0.30) if not op_nonzero.empty else 0

        group['low_hitrate'] = group['hitrate'] <= hr_cut
        group['low_op'] = group['op_per_audit'] <= op_cut

        return group

    return (
        df.groupby(['model_band', 'sel_category'])
          .apply(tag)
          .reset_index(drop=True)
    )


# ----------------------------------------------------
# STEP 3: FINAL DECISION LOGIC (NO LEAKAGE)
# ----------------------------------------------------
def assign_decision(df):

    df['decision'] = ''
    df['insight'] = ''

    for idx, row in df.iterrows():

        # 1️⃣ Low audit gate
        if row['audits'] < 15:
            df.loc[idx, ['decision','insight']] = [
                'NOT EVALUATED',
                'Audit volume below 15; results are not statistically reliable.'
            ]

        # 2️⃣ HARD FAIL — ZERO VALUE
        elif row['hitrate'] == 0 and row['op_amount'] == 0:
            df.loc[idx, ['decision','insight']] = [
                'EXCLUDE',
                'Zero findings and zero overpayment recovery indicate no audit value '
                'for this model band and selection category.'
            ]

        # 3️⃣ RELATIVE UNDERPERFORMANCE
        elif row['low_hitrate'] and row['low_op']:
            df.loc[idx, ['decision','insight']] = [
                'EXCLUDE',
                'Both hitrate and overpayment recovery are low compared to peer '
                'combinations within the same model band and selection category.'
            ]

        # 4️⃣ STRONG SIGNAL
        elif not row['low_hitrate'] or not row['low_op']:
            df.loc[idx, ['decision','insight']] = [
                'KEEP',
                'This combination demonstrates acceptable audit effectiveness or '
                'financial recovery compared to peers.'
            ]

        # 5️⃣ SAFETY NET
        else:
            df.loc[idx, ['decision','insight']] = [
                'MONITOR',
                'Performance signals are mixed; continue monitoring.'
            ]

    return df


# ----------------------------------------------------
# STEP 4: FULL PIPELINE
# ----------------------------------------------------
def evaluate_combinations(df):

    df = create_metrics(df)
    df = compute_thresholds(df)
    df = assign_decision(df)

    return df


# ----------------------------------------------------
# RUN
# ----------------------------------------------------
# df = your_dataframe
# final_result = evaluate_combinations(df)

# final_result[final_result['decision'] == 'EXCLUDE']


In [ ]:
###latest
import pandas as pd

# ----------------------------------------------------
# STEP 1: METRICS
# ----------------------------------------------------
def create_metrics(df):

    df = df.copy()
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['op_amount'] / df['audits']

    return df


# ----------------------------------------------------
# STEP 2: PEER AVERAGES (FOR SAFETY CHECK)
# ----------------------------------------------------
def add_peer_averages(df):

    peer_avg = (
        df.groupby(['model_band', 'sel_category'])
          .agg(
              peer_avg_hitrate=('hitrate', 'mean'),
              peer_avg_op=('op_per_audit', 'mean')
          )
          .reset_index()
    )

    return df.merge(peer_avg, on=['model_band', 'sel_category'], how='left')


# ----------------------------------------------------
# STEP 3: EXCLUSION LOGIC (VERY STRICT)
# ----------------------------------------------------
def assign_exclusion(df):

    df['decision'] = 'KEEP'
    df['reason'] = ''

    for idx, row in df.iterrows():

        # 1️⃣ Ignore low audit volume
        if row['audits'] < 15:
            df.loc[idx, ['decision','reason']] = [
                'NOT EVALUATED',
                'Audit volume below 15; combination not assessed.'
            ]
            continue

        # 2️⃣ Hard zero-value exclusion
        if row['hitrate'] == 0 and row['op_amount'] == 0:
            df.loc[idx, ['decision','reason']] = [
                'EXCLUDE',
                'Zero hitrate and zero overpayment recovery indicate no audit value.'
            ]
            continue

        # 3️⃣ Very poor absolute performance
        abs_poor = (
            row['hitrate'] < 0.10 and
            row['op_per_audit'] < 600
        )

        # 4️⃣ Relative underperformance vs peers
        rel_poor = (
            row['hitrate'] < 0.5 * row['peer_avg_hitrate'] and
            row['op_per_audit'] < 0.5 * row['peer_avg_op']
        )

        if abs_poor and rel_poor:
            df.loc[idx, ['decision','reason']] = [
                'EXCLUDE',
                'Extremely low hitrate and value recovery compared to both '
                'business benchmarks and peer combinations.'
            ]

    return df


# ----------------------------------------------------
# STEP 4: FULL PIPELINE
# ----------------------------------------------------
def find_poor_combinations(df):

    df = create_metrics(df)
    df = add_peer_averages(df)
    df = assign_exclusion(df)

    return df


# ----------------------------------------------------
# RUN
# ----------------------------------------------------
# df = your_dataframe
# result = find_poor_combinations(df)

# ONLY WORST COMBINATIONS:
# result[result['decision'] == 'EXCLUDE']


In [ ]:
import pandas as pd
import numpy as np

# -----------------------------
# BUSINESS CONSTANTS
# -----------------------------
MIN_AUDITS = 15
LOW_HITRATE_ABS = 0.10              # 10%
LOW_OP_PER_AUDIT = 600              # Loss floor
PEER_RELATIVE_FACTOR = 0.50         # 50% of peer performance
MAX_TOTAL_OP_TO_EXCLUDE = 100000    # Safety net

# -----------------------------
# INPUT DATAFRAME
# Expected columns:
# providertaxid, providernpi, sel_category, model_band,
# audits, findings, nofindings, op_amount
# -----------------------------
df = df.copy()

# -----------------------------
# DERIVED METRICS
# -----------------------------
df['hitrate'] = df['findings'] / df['audits']
df['op_per_audit'] = df['op_amount'] / df['audits']

# -----------------------------
# PEER BENCHMARKS (DYNAMIC)
# Peers = same sel_category + model_band
# -----------------------------
peer_stats = (
    df[df['audits'] >= MIN_AUDITS]
    .groupby(['sel_category', 'model_band'])
    .agg(
        peer_avg_hitrate=('hitrate', 'mean'),
        peer_avg_op_per_audit=('op_per_audit', 'mean')
    )
    .reset_index()
)

df = df.merge(peer_stats, on=['sel_category', 'model_band'], how='left')

# -----------------------------
# CORE CLASSIFICATION LOGIC
# -----------------------------
def classify_row(row):

    # Rule 0: insufficient data
    if row['audits'] < MIN_AUDITS:
        return 'INSUFFICIENT_DATA'

    # Rule 1: absolute zero value
    if row['hitrate'] == 0 and row['op_amount'] == 0:
        return 'EXCLUDE'

    # Dynamic peer thresholds
    hitrate_peer_threshold = row['peer_avg_hitrate'] * PEER_RELATIVE_FACTOR
    op_peer_threshold = row['peer_avg_op_per_audit'] * PEER_RELATIVE_FACTOR

    # Absolute poor checks
    abs_poor_hitrate = row['hitrate'] < LOW_HITRATE_ABS
    abs_poor_op = row['op_per_audit'] < LOW_OP_PER_AUDIT

    # Relative poor checks
    rel_poor_hitrate = row['hitrate'] < hitrate_peer_threshold
    rel_poor_op = row['op_per_audit'] < op_peer_threshold

    # Safety net: protect high value
    low_total_value = row['op_amount'] < MAX_TOTAL_OP_TO_EXCLUDE

    # Final EXCLUDE (strict, no leakage)
    if (
        abs_poor_hitrate and
        abs_poor_op and
        rel_poor_hitrate and
        rel_poor_op and
        low_total_value
    ):
        return 'EXCLUDE'

    # Borderline
    if abs_poor_hitrate or abs_poor_op:
        return 'MODERATE'

    return 'KEEP'

df['decision'] = df.apply(classify_row, axis=1)

# -----------------------------
# BUSINESS INSIGHT TEXT
# -----------------------------
def generate_insight(row):

    if row['decision'] == 'INSUFFICIENT_DATA':
        return (
            "Audit volume under this selection category and model band is below "
            "the minimum threshold, so performance cannot be reliably assessed."
        )

    if row['decision'] == 'EXCLUDE':
        return (
            "This Tin–NPI combination performs significantly worse than peers "
            "within this selection category and model band. Both hitrate and "
            "overpayment yield are materially low, resulting in poor ROI. "
            "Audit effort here is unlikely to generate meaningful value."
        )

    if row['decision'] == 'MODERATE':
        return (
            "Performance is below optimal benchmarks for this selection category "
            "and model band. Some value exists, but efficiency is weaker than peers. "
            "This combination should be deprioritized but monitored."
        )

    return (
        "This Tin–NPI combination performs in line with or better than peer "
        "benchmarks within this selection category and model band and should "
        "continue to be included."
    )

df['insight'] = df.apply(generate_insight, axis=1)

# -----------------------------
# OPTIONAL VALIDATION CHECK
# Ensures excluded OP is low
# -----------------------------
exclude_validation = (
    df[df['decision'] == 'EXCLUDE']
    .agg(
        total_audits=('audits', 'sum'),
        total_op=('op_amount', 'sum'),
        avg_hitrate=('hitrate', 'mean'),
        avg_op_per_audit=('op_per_audit', 'mean')
    )
)

print(exclude_validation)


In [ ]:
## another logic
import pandas as pd
import numpy as np

# -----------------------------
# CONFIG
# -----------------------------
MIN_AUDITS = 15
BOTTOM_PERCENTILE = 0.10     # worst 10%
MAX_TOTAL_OP_TO_EXCLUDE = 100000

df = df.copy()

# -----------------------------
# DERIVED METRICS
# -----------------------------
df['hitrate'] = df['findings'] / df['audits']
df['op_per_audit'] = df['op_amount'] / df['audits']

# -----------------------------
# FILTER VALID DATA
# -----------------------------
df['decision'] = 'KEEP'
df['insight'] = ''

valid_df = df[df['audits'] >= MIN_AUDITS].copy()

# -----------------------------
# PERCENTILE RANKING
# -----------------------------
valid_df['hitrate_pct'] = (
    valid_df
    .groupby(['sel_category', 'model_band'])['hitrate']
    .rank(pct=True, method='average')
)

valid_df['op_pct'] = (
    valid_df
    .groupby(['sel_category', 'model_band'])['op_per_audit']
    .rank(pct=True, method='average')
)

# -----------------------------
# EXCLUSION LOGIC
# -----------------------------
exclude_mask = (
    (
        (valid_df['hitrate_pct'] <= BOTTOM_PERCENTILE) &
        (valid_df['op_pct'] <= BOTTOM_PERCENTILE)
    )
    |
    (
        (valid_df['hitrate'] == 0) &
        (valid_df['op_amount'] == 0)
    )
) & (valid_df['op_amount'] < MAX_TOTAL_OP_TO_EXCLUDE)

valid_df.loc[exclude_mask, 'decision'] = 'EXCLUDE'

# -----------------------------
# MODERATE FLAG (OPTIONAL)
# -----------------------------
moderate_mask = (
    (valid_df['decision'] == 'KEEP') &
    (
        (valid_df['hitrate_pct'] <= 0.25) |
        (valid_df['op_pct'] <= 0.25)
    )
)

valid_df.loc[moderate_mask, 'decision'] = 'MODERATE'

# -----------------------------
# MERGE BACK
# -----------------------------
df.update(valid_df[['decision']])

# -----------------------------
# INSIGHT TEXT
# -----------------------------
def build_insight(row):

    if row['audits'] < MIN_AUDITS:
        return "Audit volume is too low under this selection category and model band to assess performance."

    if row['decision'] == 'EXCLUDE':
        return (
            "This Tin–NPI combination falls within the lowest performing group "
            "for both hitrate and overpayment yield within this selection category "
            "and model band. Audit effort here has consistently produced minimal value."
        )

    if row['decision'] == 'MODERATE':
        return (
            "Performance is below peer benchmarks on either hitrate or overpayment "
            "yield within this selection category and model band. This combination "
            "should be monitored and deprioritized where possible."
        )

    return (
        "This Tin–NPI combination performs in line with or better than peers "
        "within this selection category and model band."
    )

df['insight'] = df.apply(build_insight, axis=1)

# -----------------------------
# VALIDATION CHECK
# -----------------------------
print(
    df[df['decision'] == 'EXCLUDE']
    .agg(
        total_audits=('audits', 'sum'),
        total_op=('op_amount', 'sum'),
        avg_hitrate=('hitrate', 'mean'),
        avg_op_per_audit=('op_per_audit', 'mean')
    )
)


In [ ]:
##new
import pandas as pd
import numpy as np

# -----------------------------
# CONFIG
# -----------------------------
MIN_AUDITS = 15
BOTTOM_PCT = 0.10
LOW_OP_FLOOR = 600
MAX_TOTAL_OP_TO_EXCLUDE = 100000

GLOBAL_HITRATE_GOOD = 0.20
GLOBAL_OP_PER_AUDIT_GOOD = 800

df = df.copy()

# -----------------------------
# METRICS
# -----------------------------
df['hitrate'] = df['findings'] / df['audits']
df['op_per_audit'] = df['op_amount'] / df['audits']

# -----------------------------
# STAGE 1: GLOBAL PROVIDER PERFORMANCE
# -----------------------------
provider_perf = (
    df.groupby(['providertaxid', 'providernpi'])
      .agg(
          total_audits=('audits', 'sum'),
          total_findings=('findings', 'sum'),
          total_op=('op_amount', 'sum')
      )
      .reset_index()
)

provider_perf['overall_hitrate'] = (
    provider_perf['total_findings'] / provider_perf['total_audits']
)
provider_perf['overall_op_per_audit'] = (
    provider_perf['total_op'] / provider_perf['total_audits']
)

provider_perf['provider_quality'] = np.where(
    (provider_perf['overall_hitrate'] >= GLOBAL_HITRATE_GOOD) |
    (provider_perf['overall_op_per_audit'] >= GLOBAL_OP_PER_AUDIT_GOOD),
    'GOOD',
    'WEAK'
)

df = df.merge(
    provider_perf[['providertaxid', 'providernpi', 'provider_quality']],
    on=['providertaxid', 'providernpi'],
    how='left'
)

# -----------------------------
# STAGE 2: CATEGORY-LEVEL ANALYSIS
# -----------------------------
valid = df[df['audits'] >= MIN_AUDITS].copy()

valid['hitrate_pct'] = (
    valid
    .groupby(['sel_category', 'model_band'])['hitrate']
    .rank(pct=True)
)

valid['op_pct'] = (
    valid
    .groupby(['sel_category', 'model_band'])['op_per_audit']
    .rank(pct=True)
)

# Default
valid['decision'] = 'KEEP'

# EXCLUDE ONLY FOR WEAK PROVIDERS
exclude_mask = (
    (valid['provider_quality'] == 'WEAK') &
    (
        (
            (valid['hitrate'] == 0) &
            (valid['op_amount'] == 0)
        )
        |
        (
            (valid['hitrate_pct'] <= BOTTOM_PCT) &
            (valid['op_pct'] <= BOTTOM_PCT) &
            (valid['op_per_audit'] < LOW_OP_FLOOR) &
            (valid['op_amount'] < MAX_TOTAL_OP_TO_EXCLUDE)
        )
    )
)

valid.loc[exclude_mask, 'decision'] = 'EXCLUDE'

# MODERATE (optional)
moderate_mask = (
    (valid['decision'] == 'KEEP') &
    (valid['provider_quality'] == 'WEAK')
)

valid.loc[moderate_mask, 'decision'] = 'MODERATE'

# Merge back
df['decision'] = 'INSUFFICIENT_DATA'
df.loc[df['audits'] >= MIN_AUDITS, 'decision'] = valid['decision']

# -----------------------------
# INSIGHT
# -----------------------------
def insight(row):
    if row['audits'] < MIN_AUDITS:
        return "Insufficient audits to assess this combination."

    if row['decision'] == 'EXCLUDE':
        return (
            "This combination delivers consistently low financial yield and "
            "hitrate within this selection category and model band, and the "
            "provider does not perform strongly overall. Excluding this "
            "combination will reduce low-value audit effort."
        )

    if row['decision'] == 'MODERATE':
        return (
            "The provider’s overall performance is weak, but this combination "
            "does not meet strict exclusion criteria. Monitor and deprioritize."
        )

    return (
        "This combination performs acceptably or the provider performs strongly "
        "overall, so audit effort should be retained."
    )

df['insight'] = df.apply(insight, axis=1)

# -----------------------------
# VALIDATION
# -----------------------------
print(
    df[df['decision'] == 'EXCLUDE']
    .agg(
        total_audits=('audits', 'sum'),
        total_op=('op_amount', 'sum'),
        avg_hitrate=('hitrate', 'mean'),
        avg_op_per_audit=('op_per_audit', 'mean')
    )
)
